In [26]:
from gsmmutils import MyModel
import gsmmutils
gsmmutils.model.model_validator.DATA_PATH =  "../results"
import seaborn as sns
DATA_PATH = r"../data"
from os.path import join
sns.set(rc={'figure.figsize':(35,8.27)})
from cobra.flux_analysis import pfba

# Basic Simulations

1. Load the models

In [27]:
ds = MyModel(join(DATA_PATH, 'models/model_ds.xml'), 'e_Biomass__cytop')
ds.set_prism_reaction("PRISM_white_LED__extr")
print(ds.slim_optimize())
ng = MyModel(join(DATA_PATH, 'models/model_ng.xml'), 'e_Biomass__cytop')
ng.set_prism_reaction("PRISM_solar_litho__extr")
print(ng.slim_optimize())
pl = MyModel(join(DATA_PATH, 'models/model_pl.xml'), 'e_Biomass__cytop')
pl.set_prism_reaction("PRISM_fluorescent_cool_215W__extr")
pl.reactions.e_Biomass_ht__cytop.bounds = (0,0)
print(pl.slim_optimize())

Loading

Reactions: 4515
Metabolites: 3680
Genes: 1700
Model loaded
0.2006089729414475
Loading

Reactions: 4437
Metabolites: 3771
Genes: 1333
Model loaded
0.40715632401484386
Loading

Reactions: 4702
Metabolites: 4157
Genes: 1349
Model loaded
0.17297412240228494


2. Simple pFBA

In [28]:
sol = pfba(ds)
ds_original_co2 = sol.fluxes["EX_C00011__dra"]
ds.summary(sol)

Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,6.743,0,0.00%
C00009__extr,EX_C00009__dra,0.03278,0,0.00%
C00011__extr,EX_C00011__dra,8.21,1,99.70%
C00059__extr,EX_C00059__dra,0.02218,0,0.00%
C00080__extr,EX_C00080__dra,0.0558,0,0.00%
C00120__extr,EX_C00120__dra,0.0009328,10,0.11%
C00205__extr,EX_C00205__dra,448,0,0.00%
C00244__extr,EX_C00244__dra,0.5302,0,0.00%
C00305__extr,EX_C00305__dra,0.00223,0,0.00%
C00378__extr,EX_C00378__dra,0.0005361,12,0.08%


In [29]:
sol = pfba(ng)
ng_original_co2 = sol.fluxes["EX_C00011__dra"]
ng.summary()

Running pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,15.67,0,0.00%
C00009__extr,EX_C00009__dra,0.07453,0,0.00%
C00011__extr,EX_C00011__dra,19.49,1,99.91%
C00059__extr,EX_C00059__dra,0.06003,0,0.00%
C00080__extr,EX_C00080__dra,0.1344,0,0.00%
C00120__extr,EX_C00120__dra,0.0006624,10,0.03%
C00205__extr,EX_C00205__dra,634.1,0,0.00%
C00244__extr,EX_C00244__dra,1.794,0,0.00%
C00305__extr,EX_C00305__dra,0.006568,0,0.00%
C00378__extr,EX_C00378__dra,0.0003807,12,0.02%


In [30]:
sol = pfba(pl)
pl_original_co2 = sol.fluxes["EX_C00011__dra"]
pl.summary(sol)

Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,5.326,0,0.00%
C00009__extr,EX_C00009__dra,0.02683,0,0.00%
C00011__extr,EX_C00011__dra,8.01,1,99.91%
C00059__extr,EX_C00059__dra,0.0283,0,0.00%
C00080__extr,EX_C00080__dra,1.654,0,0.00%
C00120__extr,EX_C00120__dra,0.000272,10,0.03%
C00205__extr,EX_C00205__dra,872.5,0,0.00%
C00244__extr,EX_C00244__dra,1.089,0,0.00%
C00305__extr,EX_C00305__dra,0.003599,0,0.00%
C00378__extr,EX_C00378__dra,0.0001563,12,0.02%


3. Growth on different carbon sources

D. salina

In [31]:
carbon_sources = ["C00267", "C00033",  "C00116"]

with ds as model:
    print(f"Photoautotrophic: {model.slim_optimize()}")

for carbon_source in carbon_sources:
    with ds as model:
        model.reactions.NGAM__lum.bounds= (0, 1000)
        model.reactions.R01195__chlo.bounds= (-1000, 1000)
        model.exchanges.EX_C00205__dra.bounds =(0, 0)
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = ds_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print("Heterotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())
        
for carbon_source in carbon_sources:
    with ds as model:
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = ds_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print(f"Mixotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())

Photoautotrophic: 0.2006089729414475
Heterotrophic with  alpha-D-Glucose 0.05336784333264023
Heterotrophic with  Acetate 0.005652572156097585
Heterotrophic with  Glycerol 0.0752379224918814
Mixotrophic with  alpha-D-Glucose 0.40121794588284565
Mixotrophic with  Acetate 0.40121794588292065
Mixotrophic with  Glycerol 0.40121794588292065


N. gaditana

In [32]:
carbon_sources = ["C00267", "C00469", "C00181",  "C02336", "C00089", "C00962", "C00116"]

with ng as model:
    print(f"Photoautotrophic: {model.slim_optimize()}")
    
with ng as model:
    model.reactions.NGAM__lum.bounds= (0, 1000)
    model.reactions.R01195__chlo.bounds= (-1000, 1000)
    model.exchanges.EX_C00205__dra.bounds =(0, 0)
    model.exchanges.EX_C00244__dra.lower_bound = -1000
    model.exchanges.EX_C00011__dra.lower_bound = ng_original_co2
    for carbon_source in carbon_sources:
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = ng_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print("Heterotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())
      
with ng as model:
    model.reactions.NGAM__lum.bounds= (0, 1000)
    model.exchanges.EX_C00244__dra.lower_bound = -1000
    model.exchanges.EX_C00011__dra.lower_bound = ng_original_co2
    for carbon_source in carbon_sources:
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = ng_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print("Mixotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())

with ng as model:
    model.exchanges.EX_C00244__dra.lower_bound = -1
    print(f"Autotrophic with nitrate: {model.slim_optimize()}")

with ng as model:
    model.exchanges.EX_C00244__dra.lower_bound = 0
    model.exchanges.EX_C00088__dra.lower_bound = -1
    print(f"Autotrophic with nitrite: {model.slim_optimize()}")
    
with ng as model:
    model.exchanges.EX_C00244__dra.lower_bound = 0
    model.exchanges.EX_C00014__dra.lower_bound = -1
    print(f"Autotrophic with ammonia: {model.slim_optimize()}")
    
    
with ng as model:
    model.exchanges.EX_C00244__dra.lower_bound = 0
    model.exchanges.EX_C00086__dra.lower_bound = -0.5
    print(f"Autotrophic with urea: {model.slim_optimize()}")

Photoautotrophic: 0.40715632401484386
Heterotrophic with  alpha-D-Glucose 0.10215013723332139
Heterotrophic with  Ethanol 0.3797066166976865
Heterotrophic with  D-Xylose 0.6422777694155318
Heterotrophic with  beta-D-Fructose 0.8996188117257666
Heterotrophic with  Sucrose 1.150738714206511
Heterotrophic with  beta-D-Galactose 1.395702608917188
Heterotrophic with  Glycerol 1.6606789927660435
Mixotrophic with  alpha-D-Glucose 0.8143126480297167
Mixotrophic with  Ethanol 1.221468972044571
Mixotrophic with  D-Xylose 1.6286252960594194
Mixotrophic with  beta-D-Fructose 2.0172153620850017
Mixotrophic with  Sucrose 2.3020808376864665
Mixotrophic with  beta-D-Galactose 2.5823773887984887
Mixotrophic with  Glycerol 2.8797621738536407
Autotrophic with nitrate: 0.22701394887604973
Autotrophic with nitrite: 0.22701394887605647
Autotrophic with ammonia: 0.22701394887607027
Autotrophic with urea: 0.22701394887605514


P. lutheri

In [33]:
carbon_sources = ["C00267",
                  "C00033",
                  "C00116", 
                  "C00089"]

with pl as model:
    print(f"Photoautotrophic: {model.slim_optimize()}")

# Mixotrophic
for carbon_source in carbon_sources:
    with pl as model:
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = pl_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print("Mixotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())

# Heterotrophic
for carbon_source in carbon_sources:
    with pl as model:
        model.exchanges.EX_C00205__dra.bounds =(0, 0)
        model.reactions.R01195__chlo.bounds= (-1000, 1000)
        model.reactions.NGAM__lum.bounds= (0, 1000)
        pl.reactions.e_Biomass_ht__cytop.bounds = (0, 1000)
        model.objective = "e_Biomass_ht__cytop"
        model.exchanges.get_by_id(f"EX_{carbon_source}__dra").lower_bound = pl_original_co2 / model.metabolites.get_by_id(carbon_source+"__extr").elements['C']
        print("Heterotrophic with ", model.metabolites.get_by_id(carbon_source+"__extr").name, model.slim_optimize())

Photoautotrophic: 0.17297412240228494
Mixotrophic with  alpha-D-Glucose 0.3459482448045538
Mixotrophic with  Acetate 0.3459482448046104
Mixotrophic with  Glycerol 0.3459482448045529
Mixotrophic with  Sucrose 0.3459482074276978
Heterotrophic with  alpha-D-Glucose 0.039547762393365375
Heterotrophic with  Acetate 0.011805677782450317
Heterotrophic with  Glycerol 0.05789891386891362
Heterotrophic with  Sucrose 0.039547762393365576
